In [1]:
!pwd
import sys
%reload_ext autoreload
%autoreload 2
%matplotlib inline
sys.executable

C:\Users\yokoi.shingo\my_notebook\py37\ProbSpace_game_comp\notebook


'C:\\Users\\yokoi.shingo\\Anaconda3\\envs\\py37\\python.exe'

# period と A1-level を用いた A1-player 特定
- https://prob.space/topics/masato8823-Postb93d4bc4fde321a8215b

In [2]:
import os
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook as tqdm
import sys
#sys.path.append('PATH')
#from scr import config

pd.set_option("display.max_columns", 300)
pd.set_option("display.max_rows", 300)

# データの読込
train = pd.read_csv("../orig/train_data.csv")
test = pd.read_csv('../orig/test_data.csv')

train["set"] = "train"
test["set"] = "test"

data = pd.concat([train, test]).reset_index(drop=True)

## A1-level と period から A1-player を特定

<br>

### period と A1-level でソートした A1-level を シーケンスとして考えます。

<br>

### そのシーケンスを先頭から見ていき、同じレベルがくれば同じラベルをはり、レベルアップ最低試合数以上のレベルが続いた後にレベル+1のレベルがきたとき、レベルを更新して同じラベルを付与します。これをすべてのレベルに対してラベルを付与するまで繰り返します。

### https://prob.space/competitions/game_winner/discussions/takaito-Posta0ec91cc7dc4e956bac1

今回のコンペティションでは、各行が1マッチ分の情報(参加選手の武器や勝敗のデータ)を含んでいます。

<br>

そのうえで、データの出典元を少し覗くと「「A1」は投稿者であり、このプレーヤーを統計に含めると大きく偏ることに注意してください。」という注意書きがあるのがわかります。   
このことからおそらく、「A1」プレイヤーがマッチしたマッチ情報が、データセットの各行にあたります。つまり「A1」プレイヤーはある特定のプレイヤー、そのほかのA2~B4まではランダムなプレイヤーの可能性があります。 （masatoさんのtopic「A1-levelの一部抜けについて」をみてもらうとその節は有力なのではないでしょうか）

<br>

また、A1プレイヤーは特定のプレイヤーではありますが、一人ではなく複数、つまり投稿者がひとりではなく多数いるということがEDAなどしてみるとわかります。   
以上を踏まえると、仮説ではありますが、  
「A1プレイヤーの特定」が有効かもしれないとうことと、 「A1~B4プレイヤーのランダム性に基づく特徴量作成」が重要であるかもしれないという二つの仮説をつくることができます。

<br>

前者に関しては、train data と test data をconcatさせてEDAをしてみると、A1プレイヤーのみに注目した時、おそらく同一の投稿者のマッチがtrain と test の両方にいることがわかります。   
そのことから、A1プレイヤーの特定は有効なのかなという感じではあります。

<br>

後者に関しては、データセットの各行にある「A2～B4プレイヤー」はランダムで決まるので、学習データに、A2～B4プレイヤーの武器などの特徴量をそれぞれそのまま入れるとoverfitする可能性があると思います。 加えて、武器ごとの勝率などをtarget encodeing で得ようとした際には、A1プレイヤー以外のプレイヤーのデータをすべて用いて行わないと正しい勝率は得られないと考えられます。 （A1プレイヤーも含めて勝率を得たほうが、予測にはもしかしたらいいのかもしれませんが）   
今回のコンペでは、各プレイヤーの扱い方が重要になるのではと思っています。

<br>

ここに書いたのは全部私個人の感想ですが、まちがっていたら本当にすみません。

<br>

今回のコンペの課題的にランダム性がすごく大きいことが理由かどうかわりませんが、私のcvスコアもlbスコアも全然安定してません。  
cvがよくなった！と思ったらlbは下がるし..という感じです。   
結構seed値にも左右されるのではないのかなとも思っています。(私自身のスコアは、cv:0.568前後、lb:0.559前後結構幅広くという感じでした) また、学習データのoof予測値も、テストデータの予測値も、targetの分布とはことなり、1を異常に多く予測してしまいます。  
そこから、過学習しているのかなとも思っていますが、まだよくわかっていません。  
予測したラベルの分布を、targetの分布とくらべてみるのいいのかもしれません。

In [3]:
def get_seq_labels(seq, threshold=0):
    """
    seq : 時系列順のリスト
    threshold : level up のための最低試合数
    
    [3,3,3,4,4,4,4,4,7,7,7,7,2,2,2,1,1,1,2,8,8,8] : level
     => [1,1,1,1,1,1,1,1,2,2,2,2,3,3,3,4,4,4,3,2,2,2] : player id
    というように、レベルに応じてA1の特定を考えます
    
    時間の昇順に並び替えてるから、
    同じレベルなら同一playerとみなし、
    第二引数のthresholdの回数繰り返されて、レベルがインクりされてれば、同一playerと考えてるってこと
    """

    box = np.zeros(len(seq), dtype=int)  # 最終的にラベルが入るボックス
    count = 1  # label

    for _ in range(1000):
        # level : 時系列順のレベルでユニークなもの
        # s     : levelの値を格納
        
        ind = np.where(box == 0)[0][0]
        s = seq[ind]

        renew_box = []
        for i in range(len(seq)):
            if box[i] == 0:
                if s == seq[i]:
                    box[i] = count
                    renew_box.append(seq[i])
                elif (s + 1 == seq[i]) and (
                    (np.array(renew_box) == s).sum() >= threshold
                ):
                    s += 1
                    box[i] = count
            else:
                continue
        count += 1
        if (box == 0).sum() == 0:
            # box が全部埋まれば break
            break
    return box


seq = [3, 3, 3, 4, 4, 4, 4, 4, 7, 7, 7, 7, 2, 2, 2, 1, 1, 1, 2, 8, 8, 8]
box = get_seq_labels(seq)
box

array([1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 4, 4, 4, 3, 2, 2, 2])

In [4]:
sorted_data = data.sort_values(["period", "A1-level"]).reset_index(drop=True)
levels = sorted_data["A1-level"].tolist() # A1 level を時系列順にソートしたリスト

# 最低threshold回試合したらレベルアップしたと考えている
a1_player = get_seq_labels(levels, threshold=20)
a1_player

array([  1,   1,   1, ..., 314, 314, 314])

In [5]:
df = data.sort_values(["period", "A1-level"])[["set", "y", "period", "A1-level","A1-rank", "A1-weapon"]]
df["A1-player"] = a1_player
df

,set,y,period,A1-level,A1-rank,A1-weapon,A1-player
5295,train,0.0,2019-10-10T02:00:00+00:00,63,s,prime_becchu,1
15390,train,0.0,2019-10-10T02:00:00+00:00,63,s,prime_becchu,1
37652,train,0.0,2019-10-10T02:00:00+00:00,63,s,prime_becchu,1
57989,train,1.0,2019-10-10T02:00:00+00:00,63,s,prime_becchu,1
61306,train,1.0,2019-10-10T02:00:00+00:00,63,s,prime_becchu,1
...,...,...,...,...,...,...,...
65018,train,0.0,2020-01-06T00:00:00+00:00,111,x,kelvin525_deco,15
79454,test,NaN,2020-01-06T00:00:00+00:00,111,x,kelvin525_deco,15
21014,train,0.0,2020-01-06T00:00:00+00:00,401,x,furo,314
44399,train,0.0,2020-01-06T00:00:00+00:00,401,x,furo,314


In [6]:
# A1-player id = 1の人

df1 = df[df["A1-player"] == 1]
for i, _df in df1.groupby("A1-weapon"):
    _df['A1-player_win_weapon_rate'] = _df["y"].sum() / len(_df["y"].dropna())  # プレイヤーの武器ごとの勝率
    display(_df)

,set,y,period,A1-level,A1-rank,A1-weapon,A1-player,A1-player_win_weapon_rate
31335,train,0.0,2019-12-07T08:00:00+00:00,69,NaN,dualsweeper_custom,1,0.533333
35402,train,0.0,2019-12-07T08:00:00+00:00,69,NaN,dualsweeper_custom,1,0.533333
5861,train,0.0,2020-01-03T16:00:00+00:00,70,x,dualsweeper_custom,1,0.533333
17159,train,0.0,2020-01-03T16:00:00+00:00,70,x,dualsweeper_custom,1,0.533333
63055,train,0.0,2020-01-03T16:00:00+00:00,70,x,dualsweeper_custom,1,0.533333
331,train,0.0,2020-01-03T18:00:00+00:00,70,x,dualsweeper_custom,1,0.533333
1523,train,0.0,2020-01-03T18:00:00+00:00,70,x,dualsweeper_custom,1,0.533333
5993,train,1.0,2020-01-03T18:00:00+00:00,70,x,dualsweeper_custom,1,0.533333
11832,train,0.0,2020-01-03T18:00:00+00:00,70,x,dualsweeper_custom,1,0.533333
13117,train,1.0,2020-01-03T18:00:00+00:00,70,x,dualsweeper_custom,1,0.533333


,set,y,period,A1-level,A1-rank,A1-weapon,A1-player,A1-player_win_weapon_rate
3216,train,1.0,2019-12-01T04:00:00+00:00,68,s,hissen,1,1.0
27971,train,1.0,2019-12-01T04:00:00+00:00,68,s,hissen,1,1.0
78667,test,NaN,2019-12-01T04:00:00+00:00,68,s,hissen,1,1.0
90712,test,NaN,2019-12-01T04:00:00+00:00,68,s,hissen,1,1.0
25876,train,1.0,2019-12-01T06:00:00+00:00,68,a+,hissen,1,1.0


,set,y,period,A1-level,A1-rank,A1-weapon,A1-player,A1-player_win_weapon_rate
5198,train,0.0,2019-12-01T02:00:00+00:00,68,s,hissen_hue,1,0.0
91578,test,NaN,2019-12-07T08:00:00+00:00,69,NaN,hissen_hue,1,0.0
3370,train,0.0,2019-12-08T20:00:00+00:00,69,s+,hissen_hue,1,0.0


,set,y,period,A1-level,A1-rank,A1-weapon,A1-player,A1-player_win_weapon_rate
6575,train,1.0,2019-12-22T16:00:00+00:00,69,x,l3reelgun,1,0.596491
9907,train,0.0,2019-12-22T16:00:00+00:00,69,x,l3reelgun,1,0.596491
11010,train,0.0,2019-12-22T16:00:00+00:00,69,x,l3reelgun,1,0.596491
27741,train,1.0,2019-12-22T16:00:00+00:00,69,x,l3reelgun,1,0.596491
42971,train,1.0,2019-12-22T16:00:00+00:00,69,x,l3reelgun,1,0.596491
69627,test,NaN,2019-12-22T16:00:00+00:00,69,x,l3reelgun,1,0.596491
79203,test,NaN,2019-12-22T16:00:00+00:00,69,x,l3reelgun,1,0.596491
81893,test,NaN,2019-12-22T16:00:00+00:00,69,x,l3reelgun,1,0.596491
20595,train,1.0,2019-12-24T18:00:00+00:00,69,x,l3reelgun,1,0.596491
21275,train,1.0,2019-12-24T18:00:00+00:00,69,x,l3reelgun,1,0.596491


,set,y,period,A1-level,A1-rank,A1-weapon,A1-player,A1-player_win_weapon_rate
26899,train,1.0,2019-10-27T08:00:00+00:00,68,NaN,nzap85,1,0.666667
34625,train,0.0,2019-10-27T08:00:00+00:00,68,NaN,nzap85,1,0.666667
36520,train,1.0,2019-10-27T08:00:00+00:00,68,NaN,nzap85,1,0.666667
41688,train,1.0,2019-10-27T08:00:00+00:00,68,NaN,nzap85,1,0.666667
46696,train,1.0,2019-10-27T08:00:00+00:00,68,NaN,nzap85,1,0.666667
51083,train,0.0,2019-10-27T08:00:00+00:00,68,NaN,nzap85,1,0.666667
72125,test,NaN,2019-10-27T08:00:00+00:00,68,NaN,nzap85,1,0.666667
83900,test,NaN,2019-10-27T08:00:00+00:00,68,NaN,nzap85,1,0.666667
90782,test,NaN,2019-10-27T08:00:00+00:00,68,NaN,nzap85,1,0.666667


,set,y,period,A1-level,A1-rank,A1-weapon,A1-player,A1-player_win_weapon_rate
8266,train,0.0,2019-12-07T08:00:00+00:00,69,NaN,pablo,1,0.0
65348,train,0.0,2019-12-07T08:00:00+00:00,69,NaN,pablo,1,0.0


,set,y,period,A1-level,A1-rank,A1-weapon,A1-player,A1-player_win_weapon_rate
5295,train,0.0,2019-10-10T02:00:00+00:00,63,s,prime_becchu,1,0.590717
15390,train,0.0,2019-10-10T02:00:00+00:00,63,s,prime_becchu,1,0.590717
37652,train,0.0,2019-10-10T02:00:00+00:00,63,s,prime_becchu,1,0.590717
57989,train,1.0,2019-10-10T02:00:00+00:00,63,s,prime_becchu,1,0.590717
61306,train,1.0,2019-10-10T02:00:00+00:00,63,s,prime_becchu,1,0.590717
...,...,...,...,...,...,...,...,...
77634,test,NaN,2019-10-26T04:00:00+00:00,68,a+,prime_becchu,1,0.590717
7479,train,0.0,2019-10-27T08:00:00+00:00,68,NaN,prime_becchu,1,0.590717
33514,train,0.0,2019-12-07T08:00:00+00:00,69,NaN,prime_becchu,1,0.590717
58414,train,1.0,2019-12-07T08:00:00+00:00,69,NaN,prime_becchu,1,0.590717


,set,y,period,A1-level,A1-rank,A1-weapon,A1-player,A1-player_win_weapon_rate
21887,train,1.0,2019-12-01T04:00:00+00:00,68,s,screwslosher_becchu,1,0.666667
26177,train,1.0,2019-12-01T04:00:00+00:00,68,s,screwslosher_becchu,1,0.666667
69479,test,NaN,2019-12-01T04:00:00+00:00,68,s,screwslosher_becchu,1,0.666667
82933,test,NaN,2019-12-01T04:00:00+00:00,68,s,screwslosher_becchu,1,0.666667
89041,test,NaN,2019-12-01T04:00:00+00:00,68,s,screwslosher_becchu,1,0.666667
10007,train,0.0,2019-12-01T06:00:00+00:00,68,a+,screwslosher_becchu,1,0.666667
53534,train,0.0,2019-12-01T06:00:00+00:00,68,a+,screwslosher_becchu,1,0.666667
75889,test,NaN,2019-12-07T08:00:00+00:00,69,NaN,screwslosher_becchu,1,0.666667
86862,test,NaN,2019-12-07T08:00:00+00:00,69,NaN,screwslosher_becchu,1,0.666667
87312,test,NaN,2019-12-07T08:00:00+00:00,69,NaN,screwslosher_becchu,1,0.666667


,set,y,period,A1-level,A1-rank,A1-weapon,A1-player,A1-player_win_weapon_rate
56750,train,0.0,2019-12-01T02:00:00+00:00,68,s,sputtery_clear,1,0.407407
31216,train,0.0,2019-12-01T04:00:00+00:00,68,s,sputtery_clear,1,0.407407
47654,train,0.0,2019-12-01T04:00:00+00:00,68,s,sputtery_clear,1,0.407407
54252,train,1.0,2019-12-01T04:00:00+00:00,68,NaN,sputtery_clear,1,0.407407
84595,test,NaN,2019-12-01T04:00:00+00:00,68,NaN,sputtery_clear,1,0.407407
6713,train,1.0,2019-12-01T06:00:00+00:00,68,s,sputtery_clear,1,0.407407
42287,train,0.0,2019-12-01T06:00:00+00:00,68,s,sputtery_clear,1,0.407407
48756,train,0.0,2019-12-01T06:00:00+00:00,68,s,sputtery_clear,1,0.407407
62687,train,0.0,2019-12-01T06:00:00+00:00,68,s,sputtery_clear,1,0.407407
89449,test,NaN,2019-12-01T06:00:00+00:00,68,a+,sputtery_clear,1,0.407407


In [7]:
# A1-player id = 300の人

df300 = df[df["A1-player"] == 300]
for i, _df in df300.groupby("A1-weapon"):
    _df['A1-player_win_weapon_rate'] = _df["y"].sum() / len(_df["y"].dropna())  # プレイヤーの武器ごとの勝率
    display(_df)

,set,y,period,A1-level,A1-rank,A1-weapon,A1-player,A1-player_win_weapon_rate
533,train,0.0,2019-12-01T22:00:00+00:00,56,NaN,jetsweeper,300,0.285714
21202,train,1.0,2019-12-01T22:00:00+00:00,56,NaN,jetsweeper,300,0.285714
33587,train,1.0,2019-12-01T22:00:00+00:00,56,NaN,jetsweeper,300,0.285714
51918,train,0.0,2019-12-01T22:00:00+00:00,56,NaN,jetsweeper,300,0.285714
75436,test,NaN,2019-12-01T22:00:00+00:00,56,NaN,jetsweeper,300,0.285714
31599,train,0.0,2019-12-02T22:00:00+00:00,56,NaN,jetsweeper,300,0.285714
32858,train,0.0,2019-12-02T22:00:00+00:00,56,NaN,jetsweeper,300,0.285714
57052,train,0.0,2019-12-02T22:00:00+00:00,56,NaN,jetsweeper,300,0.285714
73484,test,NaN,2019-12-02T22:00:00+00:00,56,NaN,jetsweeper,300,0.285714
76929,test,NaN,2019-12-02T22:00:00+00:00,56,NaN,jetsweeper,300,0.285714


In [9]:
df[df["A1-player"] == 1]

,set,y,period,A1-level,A1-rank,A1-weapon,A1-player
5295,train,0.0,2019-10-10T02:00:00+00:00,63,s,prime_becchu,1
15390,train,0.0,2019-10-10T02:00:00+00:00,63,s,prime_becchu,1
37652,train,0.0,2019-10-10T02:00:00+00:00,63,s,prime_becchu,1
57989,train,1.0,2019-10-10T02:00:00+00:00,63,s,prime_becchu,1
61306,train,1.0,2019-10-10T02:00:00+00:00,63,s,prime_becchu,1
...,...,...,...,...,...,...,...
76149,test,NaN,2020-01-05T18:00:00+00:00,72,x,sputtery_clear,1
91914,test,NaN,2020-01-05T18:00:00+00:00,72,x,dualsweeper_custom,1
6070,train,0.0,2020-01-05T20:00:00+00:00,72,x,dualsweeper_custom,1
27995,train,1.0,2020-01-05T20:00:00+00:00,72,x,dualsweeper_custom,1


In [13]:
# A1-playerごとの勝率

import warnings
warnings.filterwarnings('ignore')

df_new = None
for p_id in tqdm(np.unique(a1_player)):
    df_player = df[df["A1-player"] == p_id]
    for i, _df in df_player.groupby("A1-player"):
        _df['A1-player_win_rate'] = _df["y"].sum() / len(_df["y"].dropna())  # 各A1プレイヤーの武器ごとの勝率

        if df_new is None:
            df_new = _df
        else:
            df_new = df_new.append(_df)
            
df_new.sort_index(ascending=True)

,set,y,period,A1-level,A1-rank,A1-weapon,A1-player,A1-player_win_rate
0,train,1.0,2019-10-15T20:00:00+00:00,139,NaN,sshooter_becchu,89,0.595573
1,train,0.0,2019-12-14T04:00:00+00:00,198,NaN,parashelter_sorella,5,0.505232
2,train,0.0,2019-12-25T14:00:00+00:00,114,a-,nzap89,123,0.502041
3,train,0.0,2019-11-11T14:00:00+00:00,336,NaN,bamboo14mk1,38,0.563584
4,train,1.0,2019-12-14T06:00:00+00:00,299,x,bold_7,36,0.540187
...,...,...,...,...,...,...,...,...
94460,test,NaN,2019-10-11T16:00:00+00:00,67,x,prime_becchu,14,0.512987
94461,test,NaN,2019-10-20T12:00:00+00:00,25,c-,longblaster_necro,77,0.549020
94462,test,NaN,2019-12-14T00:00:00+00:00,386,x,furo,133,0.496257
94463,test,NaN,2019-10-12T04:00:00+00:00,185,s+,maneuver_becchu,86,0.543478


In [14]:
# A1-playerの武器ごとの勝率

import warnings
warnings.filterwarnings('ignore')

df_new = None
for p_id in tqdm(np.unique(a1_player)):
    df_player = df[df["A1-player"] == p_id]
    
    for i, _df in df_player.groupby("A1-weapon"):
        _df['A1-player_win_weapon_rate'] = _df["y"].sum() / len(_df["y"].dropna())  # 各A1プレイヤーの武器ごとの勝率
        #display(_df)
        if df_new is None:
            df_new = _df
        else:
            df_new = df_new.append(_df)
            
df_new.sort_index(ascending=True)

,set,y,period,A1-level,A1-rank,A1-weapon,A1-player,A1-player_win_weapon_rate
0,train,1.0,2019-10-15T20:00:00+00:00,139,NaN,sshooter_becchu,89,0.666667
1,train,0.0,2019-12-14T04:00:00+00:00,198,NaN,parashelter_sorella,5,0.000000
2,train,0.0,2019-12-25T14:00:00+00:00,114,a-,nzap89,123,0.477273
3,train,0.0,2019-11-11T14:00:00+00:00,336,NaN,bamboo14mk1,38,0.000000
4,train,1.0,2019-12-14T06:00:00+00:00,299,x,bold_7,36,0.557292
...,...,...,...,...,...,...,...,...
94460,test,NaN,2019-10-11T16:00:00+00:00,67,x,prime_becchu,14,0.000000
94461,test,NaN,2019-10-20T12:00:00+00:00,25,c-,longblaster_necro,77,0.714286
94462,test,NaN,2019-12-14T00:00:00+00:00,386,x,furo,133,0.487662
94463,test,NaN,2019-10-12T04:00:00+00:00,185,s+,maneuver_becchu,86,0.500000


## 問題点・注意点
- レベルアップ最低マッチ数の設定が甘い
- 新規参入A1-playerが丁度シーケンスの対象レベルと重なったとき、同一とみなしてしまう
- レベルは不可逆、投稿者は対象期間内のすべてのマッチを投稿している仮定
- 武器などでさらに細分化するのもよさそう